In [46]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [47]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from functools import reduce
from nettoyage_donnees import *
import duckdb

con = duckdb.connect()

In [48]:
df_demographics = con.execute(f"""
    SELECT 
        icu.stay_id,
        icu.subject_id,
        icu.hadm_id,
        icu.intime,
        icu.outtime,

        adm.admission_type,
        adm.insurance,
        adm.race,
        adm.admittime,
        adm.dischtime,
        adm.hospital_expire_flag AS hospital_mortality,

        pat.gender,
        pat.anchor_age AS age

    FROM read_csv_auto('mimic-iv-3.1/icu/icustays.csv') AS icu

    LEFT JOIN read_csv_auto('mimic-iv-3.1/hosp/admissions.csv') AS adm
        ON icu.hadm_id = adm.hadm_id

    LEFT JOIN read_csv_auto('mimic-iv-3.1/hosp/patients.csv') AS pat
        ON icu.subject_id = pat.subject_id

    WHERE pat.anchor_age >= 18
""").df()

df_demographics.head()

,stay_id,subject_id,hadm_id,intime,outtime,admission_type,insurance,race,admittime,dischtime,hospital_mortality,gender,age
0,39553978,10000032,29079034,2180-07-23 14:00:00,2180-07-23 23:50:47,EW EMER.,Medicaid,WHITE,2180-07-23 12:35:00,2180-07-25 17:55:00,0,F,52
1,37081114,10000690,25860671,2150-11-02 19:37:00,2150-11-06 17:03:17,EW EMER.,Medicare,WHITE,2150-11-02 18:02:00,2150-11-12 13:45:00,0,F,86
2,39765666,10000980,26913865,2189-06-27 08:42:00,2189-06-27 20:38:27,EW EMER.,Medicare,BLACK/AFRICAN AMERICAN,2189-06-27 07:38:00,2189-07-03 03:00:00,0,F,73
3,37067082,10001217,24597018,2157-11-20 19:18:02,2157-11-21 22:08:00,EW EMER.,Private,WHITE,2157-11-18 22:56:00,2157-11-25 18:00:00,0,F,55
4,34592300,10001217,27703517,2157-12-19 15:42:24,2157-12-20 14:27:41,DIRECT EMER.,Private,WHITE,2157-12-18 16:58:00,2157-12-24 14:55:00,0,F,55


In [49]:
vent_ids = ITEMIDS["VENT_DURATION"]

modes      = ",".join(str(x) for x in vent_ids["modes"])
settings   = ",".join(str(x) for x in vent_ids["settings"])
intub      = ",".join(str(x) for x in vent_ids["intubation_events"])
extub      = ",".join(str(x) for x in vent_ids["extubation_events"])
niv        = ",".join(str(x) for x in vent_ids["noninvasive_events"])

# length of stay
sql_los = f"""
    SELECT 
        stay_id,
        subject_id,
        hadm_id,
        intime,
        outtime,
        EXTRACT(EPOCH FROM (outtime - intime)) / 3600 / 24 AS los_days
    FROM read_csv_auto('mimic-iv-3.1/icu/icustays.csv')
"""
df_los = con.execute(sql_los).df()

# ventilation duration
sql_vent = f"""
WITH raw AS (
    SELECT
        ce.stay_id,
        ce.charttime,
        ce.itemid,
        -- Define event type
        CASE 
            WHEN ce.itemid IN ({intub}) THEN 'INTUB'
            WHEN ce.itemid IN ({extub}) THEN 'EXTUB'
            WHEN ce.itemid IN ({modes}) THEN 'ON'
            WHEN ce.itemid IN ({settings}) THEN 'ON'
            WHEN ce.itemid IN ({niv}) THEN 'ON'
            ELSE NULL
        END AS event_type
    FROM read_csv_auto('mimic-iv-3.1/icu/chartevents.csv') AS ce
    WHERE ce.itemid IN (
        {modes}, {settings}, {intub}, {extub}, {niv}
    )
),

-- Mark ON/OFF transitions
events AS (
    SELECT
        stay_id,
        charttime,
        CASE 
            WHEN event_type = 'INTUB' THEN 1
            WHEN event_type = 'ON' THEN 1
            WHEN event_type = 'EXTUB' THEN 0
            ELSE NULL
        END AS vent_flag
    FROM raw
    WHERE event_type IS NOT NULL
),

-- Forward-fill vent state to get continuous segments
ff AS (
    SELECT
        stay_id,
        charttime,
        vent_flag,
        -- Build a cumulative window to detect OFF→ON transitions
        SUM(CASE WHEN vent_flag = 1 THEN 1 ELSE 0 END)
            OVER (PARTITION BY stay_id ORDER BY charttime) AS group_id
    FROM events
),

-- Compute duration inside each ON-segment
segments AS (
    SELECT
        stay_id,
        charttime,
        LEAD(charttime) OVER (PARTITION BY stay_id ORDER BY charttime) AS next_time,
        vent_flag
    FROM ff
)

SELECT
    s.stay_id,
    COALESCE(
        SUM(
            CASE 
                WHEN s.vent_flag = 1 THEN
                    EXTRACT(EPOCH FROM (COALESCE(s.next_time, icu.outtime) - s.charttime)) / 3600
                ELSE 0
            END
        ), 0
    ) AS vent_hours
FROM segments s
LEFT JOIN read_csv_auto('mimic-iv-3.1/icu/icustays.csv') AS icu
    ON s.stay_id = icu.stay_id
GROUP BY s.stay_id
"""
df_vent = con.execute(sql_vent).df()

# merge
df_los_vent = (
    df_los
    .merge(df_vent, on="stay_id", how="left")
)

df_los_vent["vent_hours"] = df_los_vent["vent_hours"].fillna(0)
df_los_vent["vent_days"]  = df_los_vent["vent_hours"] / 24

df_los_vent = df_los_vent.drop(columns=["hadm_id","subject_id","intime","outtime", "vent_hours"], errors="ignore")

df_los_vent.head()

,stay_id,los_days,vent_days
0,39553978,0.410266,0.396377
1,37081114,3.893252,3.377280
2,39765666,0.497535,0.485035
3,37067082,1.118032,1.097917
4,34592300,0.948113,0.000000


In [50]:
# Converts Python lists to comma-separated strings
def to_csv(item_list):
    return ",".join(str(x) for x in item_list)

HR      = to_csv(ITEMIDS["HR"])
RR      = to_csv(ITEMIDS["RR"])
MAP     = to_csv(ITEMIDS["MAP"])
SPO2    = to_csv(ITEMIDS["SPO2"])
FIO2    = to_csv(ITEMIDS["FIO2"])
TEMP    = to_csv(ITEMIDS["TEMP"])
GCS_E   = to_csv(ITEMIDS["GCS_E"])
GCS_V   = to_csv(ITEMIDS["GCS_V"])
GCS_M   = to_csv(ITEMIDS["GCS_M"])

PAO2_LAB   = to_csv(ITEMIDS["LABS"]["PAO2"]["lab"])
PAO2_CHART = to_csv(ITEMIDS["LABS"]["PAO2"]["chart"])

WINDOW_H = 72

# sql query
sql = f"""
WITH icu AS (
    SELECT 
        stay_id,
        intime,
        outtime
    FROM read_csv_auto('mimic-iv-3.1/icu/icustays.csv')
),

-- ---------------------------
-- 1. CHARTEVENTS within window
-- ---------------------------
ce AS (
    SELECT
        stay_id,
        charttime,
        itemid,
        valuenum,
        value
    FROM read_csv_auto('mimic-iv-3.1/icu/chartevents.csv')
    WHERE itemid IN (
        {HR}, {RR}, {MAP}, {SPO2}, {FIO2}, {TEMP},
        {GCS_E}, {GCS_V}, {GCS_M},
        {PAO2_CHART}
    )
),

ce_window AS (
    SELECT
        ce.*,
        icu.outtime,
        icu.intime,
        CASE 
            WHEN charttime BETWEEN (icu.outtime - INTERVAL '{WINDOW_H} hours')
                              AND    icu.outtime
            THEN 1 ELSE 0 END AS in_window
    FROM ce
    JOIN icu USING (stay_id)
    WHERE charttime <= icu.outtime
),

-- ---------------------------
-- 2. LABEVENTS within window
-- ---------------------------
labs AS (
    SELECT
        subject_id,
        hadm_id,
        charttime,
        itemid,
        valuenum
    FROM read_csv_auto('mimic-iv-3.1/hosp/labevents.csv')
    WHERE itemid IN ({PAO2_LAB})
),

labs_window AS (
    SELECT
        labs.*,
        icu.stay_id,
        icu.outtime,
        CASE 
            WHEN labs.charttime BETWEEN (icu.outtime - INTERVAL '{WINDOW_H} hours')
                                   AND    icu.outtime
            THEN 1 ELSE 0 END AS in_window
    FROM labs
    JOIN read_csv_auto('mimic-iv-3.1/icu/icustays.csv') AS icu
      ON labs.subject_id = icu.subject_id
     AND labs.hadm_id    = icu.hadm_id
),

-- ---------------------------
-- 3. LAST values per stay
-- ---------------------------
last_ce AS (
    SELECT
        stay_id,
        itemid,
        valuenum,
        value,
        ROW_NUMBER() OVER (PARTITION BY stay_id, itemid 
                           ORDER BY charttime DESC) AS rn
    FROM ce_window
    WHERE in_window = 1
),

last_labs AS (
    SELECT
        stay_id,
        itemid,
        valuenum,
        ROW_NUMBER() OVER (PARTITION BY stay_id, itemid 
                           ORDER BY charttime DESC) AS rn
    FROM labs_window
    WHERE in_window = 1
),

-- ---------------------------
-- 4. Pivot vitals
-- ---------------------------
pivot_ce AS (
    SELECT
        stay_id,

        MAX(CASE WHEN itemid IN ({HR})  AND rn = 1 THEN valuenum END) AS hr,
        MAX(CASE WHEN itemid IN ({RR})  AND rn = 1 THEN valuenum END) AS rr,
        MAX(CASE WHEN itemid IN ({MAP}) AND rn = 1 THEN valuenum END) AS map,
        MAX(CASE WHEN itemid IN ({SPO2}) AND rn = 1 THEN valuenum END) AS spo2,
        MAX(CASE WHEN itemid IN ({TEMP}) AND rn = 1 THEN valuenum END) AS temp,

        MAX(CASE WHEN itemid IN ({FIO2}) AND rn = 1 THEN 
                     CASE WHEN valuenum > 1 THEN valuenum/100 ELSE valuenum END 
                 END) AS fio2,

        -- GCS E and V are numeric
        MAX(CASE WHEN itemid IN ({GCS_E}) AND rn = 1 THEN valuenum END) AS gcs_e,
        MAX(CASE WHEN itemid IN ({GCS_V}) AND rn = 1 THEN valuenum END) AS gcs_v,

        -- GCS Motor requires mapping text -> numeric
        MAX(CASE WHEN itemid IN ({GCS_M}) AND rn = 1 THEN
            CASE 
                WHEN value ILIKE '%obey%' THEN 6
                WHEN value ILIKE '%local%' THEN 5
                WHEN value ILIKE '%withdraw%' THEN 4
                WHEN value ILIKE '%flex%' THEN 3
                WHEN value ILIKE '%exten%' THEN 2
                WHEN value ILIKE '%no response%' THEN 1
                WHEN value ILIKE '%none%' THEN 1
                WHEN value ILIKE '%intubat%' THEN 1
                ELSE valuenum 
            END
        END) AS gcs_m,

        -- PaO2 from chartevents fallback
        MAX(CASE WHEN itemid IN ({PAO2_CHART}) AND rn = 1 THEN valuenum END) AS pao2_chart

    FROM last_ce
    GROUP BY stay_id
),

-- ---------------------------
-- 5. Pivot labs (PaO2)
-- ---------------------------
pivot_labs AS (
    SELECT
        stay_id,
        MAX(CASE WHEN itemid IN ({PAO2_LAB}) AND rn = 1 THEN valuenum END) AS pao2_lab
    FROM last_labs
    GROUP BY stay_id
),

-- ---------------------------
-- 6. Merge and compute derived values
-- ---------------------------
merged AS (
    SELECT
        icu.stay_id,
        hr, rr, map, spo2, temp, fio2,
        gcs_e, gcs_v, gcs_m,
        pao2_lab,
        pao2_chart,

        -- GCS total
        CASE 
            WHEN gcs_e IS NOT NULL AND gcs_v IS NOT NULL AND gcs_m IS NOT NULL
            THEN gcs_e + gcs_v + gcs_m
            ELSE NULL
        END AS gcs,

        -- Final PaO2 (prefer lab)
        COALESCE(pao2_lab, pao2_chart) AS pao2,

        -- P/F ratio
        CASE 
            WHEN COALESCE(pao2_lab, pao2_chart) IS NOT NULL 
             AND fio2 IS NOT NULL AND fio2 > 0
            THEN COALESCE(pao2_lab, pao2_chart) / fio2
            ELSE NULL
        END AS pfratio

    FROM pivot_ce
    JOIN icu USING (stay_id)
    LEFT JOIN pivot_labs USING (stay_id)
)

SELECT * FROM merged
ORDER BY stay_id;
"""

df_vitals_discharge = con.execute(sql).df()
df_vitals_discharge.head()

,stay_id,hr,rr,map,spo2,temp,fio2,gcs_e,gcs_v,gcs_m,pao2_lab,pao2_chart,gcs,pao2,pfratio
0,30000153,97.0,14.0,84.0,100.0,100.1,0.4,4.0,6.0,NaN,215.0,215.0,NaN,215.0,537.500000
1,30000213,81.0,22.0,NaN,96.0,98.3,0.3,5.0,6.0,NaN,56.0,79.0,NaN,56.0,186.666667
2,30000484,95.0,22.0,NaN,100.0,99.2,NaN,4.0,6.0,NaN,21.0,NaN,NaN,21.0,NaN
3,30000646,113.0,23.0,NaN,97.0,98.0,NaN,5.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN
4,30000831,126.0,22.0,NaN,95.0,98.4,0.5,5.0,6.0,NaN,58.0,NaN,NaN,58.0,116.000000


In [51]:
def to_csv(lst):
    return ",".join(str(x) for x in lst)

modes      = to_csv(ITEMIDS["VENT_DURATION"]["modes"])
settings   = to_csv(ITEMIDS["VENT_DURATION"]["settings"])
intub_ids  = to_csv(ITEMIDS["VENT_DURATION"]["intubation_events"])
extub_ids  = to_csv(ITEMIDS["VENT_DURATION"]["extubation_events"])
niv_ids    = to_csv(ITEMIDS["VENT_DURATION"]["noninvasive_events"])

crrt_ids   = to_csv(ITEMIDS["SUPPORT"]["CRRT"])

# FAST flatten of vasopressors
pressor_ids = []
for v in ITEMIDS["PRESSORS"].values():
    if isinstance(v, list):
        pressor_ids.extend(v)
    else:
        pressor_ids.append(v)
pressor_ids_csv = ",".join(str(x) for x in pressor_ids)

WINDOW_H = 72

sql = f"""
WITH icu AS (
    SELECT stay_id, subject_id, hadm_id, intime, outtime
    FROM read_csv_auto('mimic-iv-3.1/icu/icustays.csv')
),

-- ----------------------------------------------------
-- CHARTEVENTS: Detect ventilation evidence
-- modes, settings, or intubation-related itemids
-- ----------------------------------------------------
ce AS (
    SELECT
        stay_id,
        itemid,
        charttime
    FROM read_csv_auto('mimic-iv-3.1/icu/chartevents.csv')
    WHERE itemid IN ({modes}, {settings})
),

ce_window AS (
    SELECT DISTINCT
        i.stay_id,
        CASE 
            WHEN ce.itemid IN ({modes}, {settings})
                 AND ce.charttime BETWEEN (i.outtime - INTERVAL '{WINDOW_H} hours') AND i.outtime
            THEN 1 ELSE 0 END AS vent_signal
    FROM icu i
    LEFT JOIN ce 
        ON ce.stay_id = i.stay_id
),

-- ----------------------------------------------------
-- PROCEDUREEVENTS: Intubation, extubation, CRRT
-- ----------------------------------------------------
proc AS (
    SELECT
        stay_id,
        itemid,
        endtime
    FROM read_csv_auto('mimic-iv-3.1/icu/procedureevents.csv')
    WHERE itemid IN ({intub_ids}, {extub_ids}, {crrt_ids})
),

proc_window AS (
    SELECT DISTINCT
        i.stay_id,

        -- ventilation via intubation evidence
        CASE 
            WHEN proc.itemid IN ({intub_ids})
                 AND proc.endtime BETWEEN (i.outtime - INTERVAL '{WINDOW_H} hours') AND i.outtime
            THEN 1 ELSE 0 END AS intub_signal,

        -- extubation
        CASE 
            WHEN proc.itemid IN ({extub_ids})
                 AND proc.endtime BETWEEN (i.outtime - INTERVAL '{WINDOW_H} hours') AND i.outtime
            THEN 1 ELSE 0 END AS extub_signal,

        -- CRRT
        CASE 
            WHEN proc.itemid IN ({crrt_ids})
                 AND proc.endtime BETWEEN (i.outtime - INTERVAL '{WINDOW_H} hours') AND i.outtime
            THEN 1 ELSE 0 END AS crrt_signal

    FROM icu i
    LEFT JOIN proc 
        ON proc.stay_id = i.stay_id
),

-- ----------------------------------------------------
-- INPUTEVENTS: Vasopressors
-- ----------------------------------------------------
iv AS (
    SELECT
        stay_id,
        itemid,
        starttime,
        endtime
    FROM read_csv_auto('mimic-iv-3.1/icu/inputevents.csv')
    WHERE itemid IN ({pressor_ids_csv})
),

iv_window AS (
    SELECT DISTINCT
        i.stay_id,
        CASE 
            WHEN iv.itemid IN ({pressor_ids_csv})
             AND iv.starttime <= i.outtime
             AND iv.endtime >= (i.outtime - INTERVAL '{WINDOW_H} hours')
            THEN 1 ELSE 0 END AS pressor_signal
    FROM icu i
    LEFT JOIN iv 
        ON iv.stay_id = i.stay_id
),

-- ----------------------------------------------------
-- Aggregate each signal into a stay-level indicator
-- ----------------------------------------------------
agg AS (
    SELECT
        i.stay_id,

        -- vent_last48: from modes/settings OR intubation evidence
        MAX(ce_window.vent_signal)       AS vent_signal,
        MAX(proc_window.intub_signal)    AS intub_signal,

        -- pressors
        MAX(iv_window.pressor_signal)    AS pressor_signal,

        -- extubation and CRRT
        MAX(proc_window.extub_signal)    AS extub_signal,
        MAX(proc_window.crrt_signal)     AS crrt_signal

    FROM icu i
    LEFT JOIN ce_window      USING (stay_id)
    LEFT JOIN proc_window    USING (stay_id)
    LEFT JOIN iv_window      USING (stay_id)
    GROUP BY stay_id
)

SELECT
    stay_id,

    -- same as python: if any ON signal → 1
    CASE WHEN COALESCE(vent_signal,0)=1 OR COALESCE(intub_signal,0)=1 
         THEN 1 ELSE 0 END AS vent_last48,

    COALESCE(pressor_signal,0) AS pressor_last48,
    COALESCE(extub_signal,0)   AS extubation_last48,
    COALESCE(crrt_signal,0)    AS crrt_last48

FROM agg
ORDER BY stay_id;
"""

df_supports_recent = con.execute(sql).df()
df_supports_recent.head()

,stay_id,vent_last48,pressor_last48,extubation_last48,crrt_last48
0,30000153,1,0,1,0
1,30000213,1,0,1,0
2,30000484,1,1,0,0
3,30000646,1,0,0,0
4,30000831,1,0,0,0


In [52]:
def to_csv(x):
    return ",".join(str(i) for i in x)

LABS = ITEMIDS["LABS"]

# Build CSV lists for each lab’s lab-itemids and chart-itemids
LAB_LAB_IDS = {lab: to_csv(src["lab"]) for lab, src in LABS.items()}
LAB_CHART_IDS = {lab: to_csv(src["chart"]) for lab, src in LABS.items()}

WINDOW_H = 72   # or 24, depending on your use case

sql_labs_lab = f"""
WITH icu AS (
    SELECT stay_id, subject_id, hadm_id, intime, outtime
    FROM read_csv_auto('mimic-iv-3.1/icu/icustays.csv')
),

labs_raw AS (
    SELECT
        i.stay_id,
        le.itemid,
        le.valuenum,
        le.charttime,
        ROW_NUMBER() OVER (
            PARTITION BY i.stay_id, le.itemid
            ORDER BY le.charttime DESC
        ) AS rn
    FROM read_csv_auto('mimic-iv-3.1/hosp/labevents.csv') AS le
    JOIN icu i
      ON le.subject_id = i.subject_id
     AND le.hadm_id    = i.hadm_id
    WHERE le.itemid IN ({",".join(LAB_LAB_IDS.values())})
      AND le.charttime BETWEEN (i.outtime - INTERVAL '{WINDOW_H} hours') AND i.outtime
),

pivoted AS (
    SELECT
        stay_id,
        {", ".join([
            f"MAX(CASE WHEN itemid IN ({LAB_LAB_IDS[lab]}) AND rn = 1 THEN valuenum END) AS {lab.lower()}_lab"
            for lab in LABS.keys()
        ])}
    FROM labs_raw
    GROUP BY stay_id
)

SELECT * FROM pivoted ORDER BY stay_id;
"""

df_labs_discharge = con.execute(sql_labs_lab).df()
df_labs_discharge.head()

,stay_id,lactate_lab,creatinine_lab,bun_lab,bicarb_lab,sodium_lab,potassium_lab,magnesium_lab,calcium_lab,wbc_lab,hemoglobin_lab,platelets_lab,bilirubin_lab,inr_lab,pao2_lab
0,30000153,2.1,1.1,22.0,23.0,145.0,4.8,2.2,8.0,15.2,9.8,162.0,NaN,1.1,215.0
1,30000213,0.9,3.6,61.0,24.0,139.0,4.8,2.1,8.1,5.8,7.4,226.0,0.3,NaN,56.0
2,30000484,1.6,1.2,36.0,28.0,140.0,4.0,2.2,8.3,18.3,9.2,329.0,0.3,1.3,21.0
3,30000646,NaN,0.5,8.0,23.0,139.0,4.0,2.1,7.9,9.1,13.2,171.0,NaN,1.4,NaN
4,30000831,1.4,2.0,83.0,26.0,147.0,3.1,2.9,8.8,13.0,12.7,392.0,0.7,1.3,58.0


In [53]:
LABS = ITEMIDS["LABS"]
VITAL_KEYS = ["HR", "RR", "MAP", "SPO2", "FIO2", "TEMP"]

def to_csv(lst):
    return ",".join(str(x) for x in lst)

# LABS: dict of {name: {"lab": [ids], "chart": [...]}}
LAB_IDCSV = {name: to_csv(info["lab"]) for name, info in LABS.items()}

# Vitals (from chartevents)
VITAL_IDCSV = {name.lower(): to_csv(ITEMIDS[name]) for name in VITAL_KEYS}

sql_lab_deltas = f"""
WITH icu AS (
    SELECT stay_id, subject_id, hadm_id, intime, outtime
    FROM read_csv_auto('mimic-iv-3.1/icu/icustays.csv')
),

-- 1) LABEVENTS joined to stays, restricted to 48h window
lab AS (
    SELECT
        i.stay_id,
        le.itemid,
        le.valuenum,
        le.charttime
    FROM read_csv_auto('mimic-iv-3.1/hosp/labevents.csv') AS le
    JOIN icu i
      ON le.subject_id = i.subject_id
     AND le.hadm_id    = i.hadm_id
    WHERE le.charttime BETWEEN (i.intime - INTERVAL '48 hours') AND i.outtime
),

-- 2) Last timestamp per stay/itemid
last_time AS (
    SELECT
        stay_id,
        itemid,
        MAX(charttime) AS last_time
    FROM lab
    GROUP BY stay_id, itemid
),

-- 3) Value at the last timestamp
last_val AS (
    SELECT
        l.stay_id,
        l.itemid,
        l.valuenum AS last_value,
        lt.last_time
    FROM lab l
    JOIN last_time lt
      ON l.stay_id = lt.stay_id
     AND l.itemid  = lt.itemid
     AND l.charttime = lt.last_time
),

-- 4) Candidate previous values ≥48h before last_time
prev_candidates AS (
    SELECT
        l.stay_id,
        l.itemid,
        l.valuenum,
        l.charttime,
        lv.last_time,
        ROW_NUMBER() OVER (
            PARTITION BY l.stay_id, l.itemid
            ORDER BY l.charttime DESC
        ) AS rn_desc
    FROM lab l
    JOIN last_val lv
      ON l.stay_id = lv.stay_id
     AND l.itemid  = lv.itemid
    WHERE l.charttime <= lv.last_time - INTERVAL '48 hours'
),

-- 5) Most recent candidate before cutoff
prev_val AS (
    SELECT
        stay_id,
        itemid,
        valuenum AS prev_value
    FROM prev_candidates
    WHERE rn_desc = 1
),

-- 6) Compute deltas
deltas AS (
    SELECT
        lv.stay_id,
        lv.itemid,
        (lv.last_value - pv.prev_value) AS delta_value
    FROM last_val lv
    LEFT JOIN prev_val pv
      ON lv.stay_id = pv.stay_id
     AND lv.itemid  = pv.itemid
)

-- 7) Pivot wide: delta_<labname>
SELECT
    stay_id,
    {", ".join([
        f"MAX(CASE WHEN itemid IN ({LAB_IDCSV[lab]}) THEN delta_value END) AS delta_{lab.lower()}"
        for lab in LABS.keys()
    ])}
FROM deltas
GROUP BY stay_id
ORDER BY stay_id;
"""

df_lab_deltas = con.execute(sql_lab_deltas).df()

In [54]:
sql_vital_deltas = f"""
WITH icu AS (
    SELECT stay_id, intime, outtime
    FROM read_csv_auto('mimic-iv-3.1/icu/icustays.csv')
),

char AS (
    SELECT
        ce.stay_id,
        ce.itemid,
        ce.valuenum,
        ce.charttime
    FROM read_csv_auto('mimic-iv-3.1/icu/chartevents.csv') AS ce
    JOIN icu USING (stay_id)
    WHERE ce.charttime BETWEEN (icu.intime - INTERVAL '48 hours') AND icu.outtime
      AND ce.itemid IN ({",".join(VITAL_IDCSV.values())})
),

last_time AS (
    SELECT stay_id, itemid, MAX(charttime) AS last_time
    FROM char GROUP BY stay_id, itemid
),

last_val AS (
    SELECT
        c.stay_id,
        c.itemid,
        c.valuenum AS last_value,
        lt.last_time
    FROM char c
    JOIN last_time lt
      ON c.stay_id = lt.stay_id
     AND c.itemid  = lt.itemid
     AND c.charttime = lt.last_time
),

prev_candidates AS (
    SELECT
        c.stay_id,
        c.itemid,
        c.valuenum,
        c.charttime,
        lv.last_time,
        ROW_NUMBER() OVER (
            PARTITION BY c.stay_id, c.itemid
            ORDER BY c.charttime DESC
        ) AS rn_desc
    FROM char c
    JOIN last_val lv USING (stay_id, itemid)
    WHERE c.charttime <= lv.last_time - INTERVAL '48 hours'
),

prev_val AS (
    SELECT stay_id, itemid, valuenum AS prev_value
    FROM prev_candidates
    WHERE rn_desc = 1
),

deltas AS (
    SELECT
        lv.stay_id,
        lv.itemid,
        (lv.last_value - pv.prev_value) AS delta_value
    FROM last_val lv
    LEFT JOIN prev_val pv USING (stay_id, itemid)
)

SELECT
    stay_id,
    {", ".join([
        f"MAX(CASE WHEN itemid IN ({VITAL_IDCSV[vital.lower()]}) THEN delta_value END) AS delta_{vital.lower()}"
        for vital in VITAL_KEYS
    ])}
FROM deltas
GROUP BY stay_id
ORDER BY stay_id;
"""

df_vitals_deltas = con.execute(sql_vital_deltas).df()

In [55]:
df_trajectories_48h = (
    df_lab_deltas
    .merge(df_vitals_deltas, on="stay_id", how="left")
)

df_trajectories_48h.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93227 entries, 0 to 93226
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   stay_id           93227 non-null  int64  
 1   delta_lactate     14690 non-null  float64
 2   delta_creatinine  43423 non-null  float64
 3   delta_bun         43392 non-null  float64
 4   delta_bicarb      43169 non-null  float64
 5   delta_sodium      43582 non-null  float64
 6   delta_potassium   43851 non-null  float64
 7   delta_magnesium   41865 non-null  float64
 8   delta_calcium     40179 non-null  float64
 9   delta_wbc         42424 non-null  float64
 10  delta_hemoglobin  42512 non-null  float64
 11  delta_platelets   42437 non-null  float64
 12  delta_bilirubin   16197 non-null  float64
 13  delta_inr         31643 non-null  float64
 14  delta_pao2        16826 non-null  float64
 15  delta_hr          43748 non-null  float64
 16  delta_rr          43520 non-null  float6

In [56]:
# Antibiotics dictionary keys → regex pattern
antibiotic_regex = "|".join(ITEMIDS["ANTIBIOTICS"].keys())

sedative_regex = "|".join([
    "midazolam",
    "lorazepam",
    "propofol",
    "dexmed", "dexme",
    "ketamine"
])

steroid_regex = "|".join([
    "hydrocortisone",
    "methylpred", "methylprednisolone"
])

diuretic_regex = "|".join([
    "furosemide",
    "bumetanide",
    "acetazolamide"
])

# Flatten vasopressor itemids
pressor_itemids = []
for v in ITEMIDS["PRESSORS"].values():
    if isinstance(v, list):
        pressor_itemids.extend(v)
    else:
        pressor_itemids.append(v)

pressor_ids_csv = ",".join(str(x) for x in pressor_itemids)

WINDOW_H = 72

sql_meds = f"""
WITH icu AS (
    SELECT stay_id, hadm_id, outtime
    FROM read_csv_auto('mimic-iv-3.1/icu/icustays.csv')
),

-- =========================================
-- 1) VASOPRESSORS FROM INPUTEVENTS
-- =========================================
iv AS (
    SELECT
        i.stay_id,
        CASE 
            WHEN ie.itemid IN ({pressor_ids_csv})
             AND ie.starttime <= i.outtime
             AND ie.endtime   >= (i.outtime - INTERVAL '{WINDOW_H} hours')
            THEN 1 ELSE 0 END AS pressor_signal
    FROM icu i
    LEFT JOIN read_csv_auto('mimic-iv-3.1/icu/inputevents.csv') AS ie
        ON ie.stay_id = i.stay_id
),

iv_agg AS (
    SELECT stay_id, MAX(pressor_signal) AS pressor_last72
    FROM iv
    GROUP BY stay_id
),

-- =========================================
-- 2) PRESCRIPTIONS (RX MEDICATIONS)
-- =========================================
rx AS (
    SELECT
        i.stay_id,
        p.drug
    FROM icu i
    JOIN read_csv_auto('mimic-iv-3.1/hosp/prescriptions.csv') AS p
        ON p.hadm_id = i.hadm_id
    WHERE p.starttime BETWEEN (i.outtime - INTERVAL '{WINDOW_H} hours') AND i.outtime
),

rx_agg AS (
    SELECT
        stay_id,
        MAX(CASE WHEN REGEXP_MATCHES(LOWER(drug), '{antibiotic_regex}') THEN 1 ELSE 0 END) AS antibiotics_last72,
        MAX(CASE WHEN REGEXP_MATCHES(LOWER(drug), '{sedative_regex}')    THEN 1 ELSE 0 END) AS sedatives_last72,
        MAX(CASE WHEN REGEXP_MATCHES(LOWER(drug), '{steroid_regex}')     THEN 1 ELSE 0 END) AS steroids_last72,
        MAX(CASE WHEN REGEXP_MATCHES(LOWER(drug), '{diuretic_regex}')    THEN 1 ELSE 0 END) AS diuretics_last72
    FROM rx
    GROUP BY stay_id
)

-- =========================================
-- 3) COMBINE BOTH SOURCES
-- =========================================
SELECT
    i.stay_id,

    -- Flags default to 0 when missing
    COALESCE(iv_agg.pressor_last72, 0)        AS pressor_last72,
    COALESCE(rx_agg.antibiotics_last72, 0)    AS antibiotics_last72,
    COALESCE(rx_agg.sedatives_last72, 0)      AS sedatives_last72,
    COALESCE(rx_agg.steroids_last72, 0)       AS steroids_last72,
    COALESCE(rx_agg.diuretics_last72, 0)      AS diuretics_last72

FROM icu i
LEFT JOIN iv_agg USING (stay_id)
LEFT JOIN rx_agg USING (stay_id)
ORDER BY stay_id;
"""

df_meds_recent = con.execute(sql_meds).df()
df_meds_recent.head()

,stay_id,pressor_last72,antibiotics_last72,sedatives_last72,steroids_last72,diuretics_last72
0,30000153,0,0,1,0,0
1,30000213,0,0,1,0,1
2,30000484,1,1,0,0,0
3,30000646,0,1,0,0,0
4,30000831,0,1,1,0,1


In [57]:
sql_operational = f"""
WITH icu AS (
    SELECT stay_id, outtime
    FROM read_csv_auto('mimic-iv-3.1/icu/icustays.csv')
)

SELECT
    stay_id,

    -- night discharge (20:00–23:59 and 00:00–07:59)
    CASE
        WHEN HOUR(outtime) BETWEEN 20 AND 23
          OR HOUR(outtime) BETWEEN 0 AND 7
        THEN 1 ELSE 0
    END AS night_discharge,

    -- weekend discharge (Saturday=6, Sunday=7)
    CASE
        WHEN DAYOFWEEK(outtime) IN (6,7)
        THEN 1 ELSE 0
    END AS weekend_discharge

FROM icu
ORDER BY stay_id;
"""
df_operational = con.execute(sql_operational).df()
df_operational.head()

,stay_id,night_discharge,weekend_discharge
0,30000153,1,1
1,30000213,1,0
2,30000484,1,0
3,30000646,0,1
4,30000831,0,0


In [58]:
con.execute("CREATE OR REPLACE TABLE demographics AS SELECT * FROM df_demographics")
con.execute("CREATE OR REPLACE TABLE vitals_discharge AS SELECT * FROM df_vitals_discharge")
con.execute("CREATE OR REPLACE TABLE supports_recent AS SELECT * FROM df_supports_recent")
con.execute("CREATE OR REPLACE TABLE labs_discharge AS SELECT * FROM df_labs_discharge")
con.execute("CREATE OR REPLACE TABLE trajectories_48h AS SELECT * FROM df_trajectories_48h")
con.execute("CREATE OR REPLACE TABLE meds_recent AS SELECT * FROM df_meds_recent")
con.execute("CREATE OR REPLACE TABLE operational AS SELECT * FROM df_operational")
con.execute("CREATE OR REPLACE TABLE los_vent AS SELECT * FROM df_los_vent")

In [59]:
def load_mimic_duckdb(
    con,
    table_name,
    dataset="demo",      # "demo" or "full"
    section="hosp",      # "hosp" or "icu"
    base_path="/Users/qili/Library/CloudStorage/OneDrive-SanteetServicessociaux/Soins intensifs/Érudition/Recherche/MSc MedComp/MMD6020 Fondements médecine computationnelle/Projet"  
):
    """
    Load a MIMIC-IV table directly into DuckDB.
    Automatically infers dates and uses safe typing.
    Creates a DuckDB table named <table_name>.
    """

    folder = {
        "demo": "mimic-iv-clinical-database-demo-2.2",
        "full": "mimic-iv-3.1"
    }[dataset]

    file_path = os.path.join(base_path, folder, section, f"{table_name}.csv")

    print(f"→ Loading {table_name} from: {file_path}")

    # Use DuckDB’s fast CSV reader
    con.execute(f"""
        CREATE OR REPLACE TABLE {table_name} AS
        SELECT * FROM read_csv_auto('{file_path}', SAMPLE_SIZE=-1);
    """)

    print(f"→ Table created: {table_name}")

In [60]:
df_diagnoses_icd = load_mimic_duckdb(con, "diagnoses_icd", dataset="full", section="hosp")

sql_charlson = """
WITH dx AS (
    SELECT
        hadm_id,
        icd_version,
        UPPER(
            CASE
                WHEN icd_version = 10 THEN SUBSTR(icd_code, 1, 3)
                WHEN icd_version = 9  THEN SUBSTR(SPLIT_PART(icd_code, '.', 1), 1, 3)
            END
        ) AS icd_prefix
    FROM diagnoses_icd
),

charlson_raw AS (
    SELECT
        hadm_id,

        -- 1-point conditions
        MAX(CASE WHEN (icd_version=9  AND icd_prefix ~ '^(410|412)$')
                  OR (icd_version=10 AND icd_prefix ~ '^(I21|I22|I25)$') THEN 1 ELSE 0 END) AS mi,

        MAX(CASE WHEN (icd_version=9  AND icd_prefix ~ '^(398|402|404|428)$')
                  OR (icd_version=10 AND icd_prefix ~ '^(I50|I11|I13)$') THEN 1 ELSE 0 END) AS chf,

        MAX(CASE WHEN (icd_version=9  AND icd_prefix ~ '^(440|441|443|447|557|V43)$')
                  OR (icd_version=10 AND icd_prefix ~ '^(I70|I71|I73|I77|I79|K55|Z95)$') THEN 1 ELSE 0 END) AS pvd,

        MAX(CASE WHEN (icd_version=9  AND icd_prefix ~ '^(430|431|432|433|434|435|436|437|438)$')
                  OR (icd_version=10 AND icd_prefix ~ '^(I60|I61|I62|I63|I64|I65|I66|I67|I68|I69|G45|G46)$') THEN 1 ELSE 0 END) AS cerebro,

        MAX(CASE WHEN (icd_version=9  AND icd_prefix='290')
                  OR (icd_version=10 AND icd_prefix ~ '^(F00|F01|F02|F03|G30)$') THEN 1 ELSE 0 END) AS dementia,

        MAX(CASE WHEN (icd_version=9  AND icd_prefix ~ '^(490|491|492|493|494|495|496)$')
                  OR (icd_version=10 AND icd_prefix ~ '^(J40|J41|J42|J43|J44|J45|J46|J47|J6.$)') THEN 1 ELSE 0 END) AS copd,

        MAX(CASE WHEN (icd_version=9  AND icd_prefix ~ '^(446|701|710|711|714|719)$')
                  OR (icd_version=10 AND icd_prefix ~ '^(M05|M06|M31|M32|M33|M34|M35|M36)$') THEN 1 ELSE 0 END) AS rheum,

        MAX(CASE WHEN (icd_version=9  AND icd_prefix ~ '^(531|532|533|534)$')
                  OR (icd_version=10 AND icd_prefix ~ '^(K25|K26|K27|K28)$') THEN 1 ELSE 0 END) AS pud,

        MAX(CASE WHEN (icd_version=9  AND icd_prefix ~ '^(570|571)$')
                  OR (icd_version=10 AND icd_prefix ~ '^(B18|K70|K71|K72|K73|K74)$') THEN 1 ELSE 0 END) AS mild_liver,

        MAX(CASE WHEN (icd_version=9  AND icd_prefix='250')
                   OR (icd_version=10 AND icd_prefix ~ '^(E10|E11|E12|E13|E14)$') THEN 1 ELSE 0 END) AS diabetes,

        -- 2-point conditions
        MAX(CASE WHEN (icd_version=9  AND icd_prefix ~ '^(342|343|344)$')
                  OR (icd_version=10 AND icd_prefix ~ '^(G81|G82|G04)$') THEN 1 ELSE 0 END) AS hemiplegia,

        MAX(CASE WHEN (icd_version=9  AND icd_prefix ~ '^(582|583|585|586|588)$')
                  OR (icd_version=10 AND icd_prefix ~ '^(N18|N19|N05|N25|Z49|Z94|Z99)$') THEN 1 ELSE 0 END) AS renal,

        MAX(CASE WHEN (icd_version=9  AND icd_prefix ~ '^(140|141|142|143|144|145|146|147|148|149|150|151|152|153|154|155|156|157|158|159|160|161|162|163|164|165|170|171|172|174|175|176)$')
                  OR (icd_version=10 AND icd_prefix LIKE 'C%') THEN 1 ELSE 0 END) AS malignancy,

        MAX(CASE WHEN (icd_version=9  AND icd_prefix='572')
                  OR (icd_version=10 AND icd_prefix ~ '^(I85|I86|I98|K70|K71|K72)$') THEN 1 ELSE 0 END) AS severe_liver,

        -- 6-point conditions
        MAX(CASE WHEN (icd_version=9  AND icd_prefix ~ '^(196|197|198|199)$')
                  OR (icd_version=10 AND icd_prefix ~ '^(C77|C78|C79|C80)$') THEN 1 ELSE 0 END) AS metastatic,

        MAX(CASE WHEN (icd_version=9  AND icd_prefix ~ '^(042|043|044)$')
                  OR (icd_version=10 AND icd_prefix ~ '^(B20|B21|B22|B24)$') THEN 1 ELSE 0 END) AS hiv

    FROM dx
    GROUP BY hadm_id
),

charlson AS (
    SELECT
        hadm_id,
        -- Apply weights
        (mi * 1) +
        (chf * 1) +
        (pvd * 1) +
        (cerebro * 1) +
        (dementia * 1) +
        (copd * 1) +
        (rheum * 1) +
        (pud * 1) +
        (mild_liver * 1) +
        (diabetes * 1) +
        (hemiplegia * 2) +
        (renal * 2) +
        (malignancy * 2) +
        (severe_liver * 3) +
        (metastatic * 6) +
        (hiv * 6)
        AS charlson_index
    FROM charlson_raw
)

SELECT * FROM charlson;
"""

df_charlson = con.execute(sql_charlson).df()
df_charlson.head()

con.register("charlson", df_charlson)

→ Loading diagnoses_icd from: /Users/qili/Library/CloudStorage/OneDrive-SanteetServicessociaux/Soins intensifs/Érudition/Recherche/MSc MedComp/MMD6020 Fondements médecine computationnelle/Projet/mimic-iv-3.1/hosp/diagnoses_icd.csv
→ Table created: diagnoses_icd


In [61]:
icustays = load_mimic_duckdb(con, "icustays", dataset="full", section="icu")

con.execute("PRAGMA table_info('icustays')").df()

→ Loading icustays from: /Users/qili/Library/CloudStorage/OneDrive-SanteetServicessociaux/Soins intensifs/Érudition/Recherche/MSc MedComp/MMD6020 Fondements médecine computationnelle/Projet/mimic-iv-3.1/icu/icustays.csv
→ Table created: icustays


,cid,name,type,notnull,dflt_value,pk
0,0,subject_id,BIGINT,False,None,False
1,1,hadm_id,BIGINT,False,None,False
2,2,stay_id,BIGINT,False,None,False
3,3,first_careunit,VARCHAR,False,None,False
4,4,last_careunit,VARCHAR,False,None,False
5,5,intime,TIMESTAMP,False,None,False
6,6,outtime,TIMESTAMP,False,None,False
7,7,los,DOUBLE,False,None,False


In [62]:
sql_create_readmit = """
CREATE OR REPLACE TABLE readmit_72h AS
WITH ordered AS (
    SELECT
        stay_id,
        subject_id,
        hadm_id,
        intime,
        outtime,
        LEAD(intime) OVER (
            PARTITION BY subject_id, hadm_id
            ORDER BY intime
        ) AS next_intime
    FROM icustays
),
flagged AS (
    SELECT
        stay_id,
        CASE
            WHEN next_intime IS NOT NULL
                 AND next_intime >= outtime
                 AND next_intime <= outtime + INTERVAL 72 HOUR
            THEN 1
            ELSE 0
        END AS readmit_72h
    FROM ordered
)
SELECT * FROM flagged;
"""

con.execute(sql_create_readmit)
df_readmit = con.execute("SELECT * FROM readmit_72h ORDER BY stay_id").df()
df_readmit.head()

,stay_id,readmit_72h
0,30000153,0
1,30000213,0
2,30000484,0
3,30000646,1
4,30000831,1


In [63]:
con.execute("DROP TABLE IF EXISTS demographics;")
con.execute("CREATE TABLE demographics AS SELECT * FROM df_demographics;")

In [64]:
sql_build_final = """
WITH base AS (
    SELECT stay_id FROM demographics
),

final AS (
    SELECT
        b.stay_id,

        -- 1) DEMOGRAPHICS
        d.subject_id,
        d.gender,
        d.age,
        d.race,
        d.insurance,
        d.admission_type,
        d.admittime,
        d.dischtime,
        d.intime,
        d.hospital_mortality,

        -- 2) VITALS AT DISCHARGE
        vd.hr,
        vd.rr,
        vd.map,
        vd.spo2,
        vd.temp,
        vd.fio2,
        vd.gcs,
        vd.pao2,
        vd.pfratio,

        -- 3) SUPPORTS
        sr.vent_last48,
        sr.pressor_last48,
        sr.extubation_last48,
        sr.crrt_last48,

        -- 4) LABS
        ld.* EXCLUDE(stay_id),

        -- 5) TRAJECTORIES
        t.* EXCLUDE(stay_id),

        -- 6) MEDS
        mr.pressor_last72,
        mr.antibiotics_last72,
        mr.sedatives_last72,
        mr.steroids_last72,
        mr.diuretics_last72,

        -- 7) OPERATIONAL
        op.night_discharge,
        op.weekend_discharge,

        -- 8) LOS
        lv.los_days,
        lv.vent_days,

        -- 9) CHARLSON
        COALESCE(c.charlson_index, 0) AS charlson_index,

        -- 10) READMISSION
        COALESCE(rh.readmit_72h, 0) AS readmit_72h

    FROM base b
    LEFT JOIN demographics       d  USING (stay_id)
    LEFT JOIN charlson           c  USING (hadm_id)
    LEFT JOIN vitals_discharge   vd USING (stay_id)
    LEFT JOIN supports_recent    sr USING (stay_id)
    LEFT JOIN labs_discharge     ld USING (stay_id)
    LEFT JOIN trajectories_48h   t  USING (stay_id)
    LEFT JOIN meds_recent        mr USING (stay_id)
    LEFT JOIN operational        op USING (stay_id)
    LEFT JOIN los_vent           lv USING (stay_id)
    LEFT JOIN readmit_72h        rh USING (stay_id)
)

-- Filter out <48h mechanical ventilation
SELECT *
FROM final
WHERE vent_days >= 2
ORDER BY stay_id;
"""

df_final = con.execute(sql_build_final).df()
df_final.head()

,stay_id,subject_id,gender,age,race,insurance,admission_type,admittime,dischtime,intime,...,antibiotics_last72,sedatives_last72,steroids_last72,diuretics_last72,night_discharge,weekend_discharge,los_days,vent_days,charlson_index,readmit_72h
0,30000484,18421337,M,91,MULTIPLE RACE/ETHNICITY,Medicare,EW EMER.,2136-01-14 17:22:00,2136-01-24 16:00:00,2136-01-14 17:23:32,...,1,0,0,0,1,0,2.478889,2.421620,3,0
1,30000646,12207593,M,43,ASIAN - CHINESE,Medicaid,EW EMER.,2194-04-27 18:43:00,2194-05-06 02:29:00,2194-04-29 01:39:22,...,1,0,0,0,0,1,4.697523,4.697083,8,1
2,30000831,15726459,M,78,WHITE,Private,URGENT,2140-04-17 21:25:00,2140-05-18 21:00:00,2140-04-17 21:26:33,...,1,1,0,1,0,0,2.705139,2.598576,6,1
3,30001336,12168737,M,77,UNKNOWN,None,URGENT,2186-03-20 00:44:00,2186-03-22 19:10:00,2186-03-20 00:44:48,...,0,0,0,1,0,0,2.778426,2.772731,5,0
4,30001471,17461994,M,86,HISPANIC/LATINO - SALVADORAN,Medicaid,OBSERVATION ADMIT,2189-05-02 05:43:00,2189-05-05 15:38:00,2189-05-02 06:57:00,...,0,0,0,0,0,0,3.378854,3.210104,3,0


In [65]:
df_clean = clean_df(df_final)
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39175 entries, 0 to 39174
Data columns (total 61 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   stay_id             39175 non-null  int64         
 1   insurance           38644 non-null  object        
 2   race                39175 non-null  object        
 3   gender              39175 non-null  object        
 4   age                 39175 non-null  int64         
 5   spo2                39121 non-null  float64       
 6   fio2                24514 non-null  float64       
 7   temp                39009 non-null  float64       
 8   map                 15990 non-null  float64       
 9   gcs                 1653 non-null   float64       
 10  pao2                22944 non-null  float64       
 11  pfratio             17986 non-null  float64       
 12  vent_last48         39175 non-null  int32         
 13  pressor_last48      39175 non-null  int32     

In [66]:
table1 = generate_table1(df_final)
table1

,Variable,Non-readmitted (n=36786),Readmitted (n=2389),P-value
0,age,64.19 ± 15.91,63.89 ± 15.47,0.370
1,female,0: 20935 (56.9%); 1: 15851 (43.1%),0: 1378 (57.7%); 1: 1011 (42.3%),0.474
2,los_days,6.72 ± 7.23,7.13 ± 6.99,0.006
3,vent_days,6.60 ± 7.45,7.00 ± 6.96,0.006
4,pressor_last48,0: 26006 (70.7%); 1: 10780 (29.3%),0: 1745 (73.0%); 1: 644 (27.0%),0.015
5,crrt_last48,0: 34435 (93.6%); 1: 2351 (6.4%),0: 2213 (92.6%); 1: 176 (7.4%),0.066
6,hospital_mortality,0: 30818 (83.8%); 1: 5968 (16.2%),0: 1902 (79.6%); 1: 487 (20.4%),<0.001
